In [1]:
from pathlib import Path
from random import randint
from pydantic import BaseModel
from astrbot_config import AstrbotConfigEntry


class SubConfigExample(BaseModel):
    detail: str
    enabled: bool

class ConfigExample(BaseModel):
    name: str
    value: int
    sub_config: SubConfigExample

AstrbotConfigEntry[ConfigExample].model_rebuild()

cfg = AstrbotConfigEntry[ConfigExample].bind(
    group = "A",
    name = "example_config",
    default = ConfigExample(
        name="default_name",
        value=42,
        sub_config=SubConfigExample(
            detail="default_detail",
            enabled=True
        )
    ),
    description = "An example configuration entry for demonstration purposes.",
    cfg_dir = Path.cwd() / "config_examples"
)


# print(cfg.load())  # Load the configuration from file or use default
print(cfg.value)  # Access the current configuration value

cfg.value.name = "new_name" + str(randint(1, 100))  # Modify the configuration

cfg.save()  # Save the updated configuration back to file

print(cfg.value)  # Verify the saved configuration


name='new_name67' value=42 sub_config=SubConfigExample(detail='default_detail', enabled=True)
name='new_name28' value=42 sub_config=SubConfigExample(detail='default_detail', enabled=True)


# 密钥

In [ ]:
import importlib
import astrbot_config.config
importlib.reload(astrbot_config.config)

In [ ]:
# 展示密钥安全保存
from pathlib import Path
from astrbot_config import AstrbotSecretKey,AstrbotConfigEntry

from astrbot_canary_api import IAstrbotConfigEntry
from pydantic import BaseModel, Field

class ModelConfig(BaseModel):
    base_url: str = "xxx"
    api_key: AstrbotSecretKey = Field(default_factory=lambda: AstrbotSecretKey(key_name="gpt5-api-key"))


cfg_api_key: IAstrbotConfigEntry[ModelConfig] = AstrbotConfigEntry[ModelConfig].bind(
    group="A",
    name="api_key",
    default=ModelConfig(),
    description="",
    cfg_dir =Path().cwd() / "config_examples"
)

# 直接打印，不会泄露实际的key
print(cfg_api_key.value.api_key)
cfg_api_key.save()

# 这里模拟CLI程序设置secret

# 记得保存save() 不然下次启动无法自动加载
# 请勿像这里一样硬编码！
cfg_api_key.value.api_key.secret = "sk-d4a5d4a54dccad5..."
cfg_api_key.save()
# 
# 建议with语句读取密钥 好吧基本只能这样获取
async with cfg_api_key.value.api_key.actx() as s:
    print(s)

with cfg_api_key.value.api_key.ctx() as s:
    print(s)


print("*"*10)
print(cfg_api_key.value.api_key)
# print(cfg_api_key.value.api_key.secret) 这里会警告不要直接访问secret属性
print(cfg_api_key.value.api_key)

In [ ]:
# 检查密钥库，删掉刚刚测试的密钥
import keyring

# 删除当前使用的密钥
try:
    keyring.delete_password("astrbot", "@astrbot:gpt5-api-key")
    print("已删除密钥: astrbot / @astrbot:gpt5-api-key")
except Exception as e:
    print(f"删除失败，可能不存在: {e}")

# 注意：keyring 库不支持按服务名批量删除密钥
# 每个密钥需要具体的 service 和 key_id
# 之前的测试使用了随机 UUID 作为 key_id，无法自动识别

# 手动清理所有 astrbot 相关的密钥（Windows 示例）
# 在 Windows 上：
# 1. 搜索 "Credential Manager" 打开凭据管理器
# 2. 转到 "Windows Credentials"
# 3. 查找以 "astrbot" 开头的凭据，删除它们
# 或者使用命令行： rundll32.exe keymgr.dll, KRShowKeyMgr

print("请手动清理系统密钥库中的所有 'astrbot' 相关凭据。keyring 不支持按服务名批量删除。")